In [1]:
import os
import torch
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np
import cv2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

############################
# 모델 구조 정의 (학습시 사용한 것과 동일 - U-Net 스킵 구조)
############################

class GatedConv2d(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, activation=torch.nn.ReLU()):
        super().__init__()
        self.feature_conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.mask_conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.sigmoid = torch.nn.Sigmoid()
        self.activation = activation

    def forward(self, x):
        f = self.feature_conv(x)
        m = self.mask_conv(x)
        gated = self.sigmoid(m)
        if self.activation is not None:
            f = self.activation(f)
        return f * gated

class GatedDeconv2d(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=4, stride=2, padding=1, activation=torch.nn.ReLU()):
        super().__init__()
        self.feature_deconv = torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding)
        self.mask_deconv = torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding)
        self.sigmoid = torch.nn.Sigmoid()
        self.activation = activation

    def forward(self, x):
        f = self.feature_deconv(x)
        m = self.mask_deconv(x)
        gated = self.sigmoid(m)
        if self.activation is not None:
            f = self.activation(f)
        return f * gated

class ContextualAttention(torch.nn.Module):
    def __init__(self, kernel_size=3, stride=1, dilation=1):
        super().__init__()
        self.conv = torch.nn.Conv2d(512, 512, kernel_size, stride, dilation, bias=False)
        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, x):
        B,C,H,W = x.size()
        query = x.view(B,C,-1)
        key = x.view(B,C,-1)
        value = x.view(B,C,-1)
        attn = torch.bmm(query.permute(0,2,1), key)  # (B,H*W,C) x (B,C,H*W)
        attn = self.softmax(attn)
        out = torch.bmm(attn, value.permute(0,2,1))
        out = out.permute(0,2,1).view(B,C,H,W)
        out = self.conv(out)
        return out

###############
# Stage1Generator (U-Net 스킵 구조)
###############
class Stage1Generator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # ----- Encoder (Down) -----
        self.enc1 = GatedConv2d(4, 64, 4, 2, 1)
        self.enc2 = GatedConv2d(64, 128, 4, 2, 1)
        self.enc3 = GatedConv2d(128, 256, 4, 2, 1)
        self.enc4 = GatedConv2d(256, 512, 4, 2, 1)

        # ----- Decoder (Up) -----
        self.dec1_up = GatedDeconv2d(512, 256, 4, 2, 1)
        self.dec1_conv = GatedConv2d(256 + 256, 256, 3, 1, 1)

        self.dec2_up = GatedDeconv2d(256, 128, 4, 2, 1)
        self.dec2_conv = GatedConv2d(128 + 128, 128, 3, 1, 1)

        self.dec3_up = GatedDeconv2d(128, 64, 4, 2, 1)
        self.dec3_conv = GatedConv2d(64 + 64, 64, 3, 1, 1)

        self.dec4_up = GatedDeconv2d(64, 64, 4, 2, 1)
        self.dec4_conv = torch.nn.Conv2d(64, 3, 3, 1, 1)
        self.final_act = torch.nn.Sigmoid()

    def forward(self, x, mask):
        # Encoder
        inp = torch.cat((x, mask), dim=1)  # (N,4,H,W)
        e1 = self.enc1(inp)    # (N,64,  H/2,  W/2)
        e2 = self.enc2(e1)     # (N,128, H/4,  W/4)
        e3 = self.enc3(e2)     # (N,256, H/8,  W/8)
        e4 = self.enc4(e3)     # (N,512, H/16, W/16)

        # Decoder (Skip Connection)
        d1_up = self.dec1_up(e4)                    # (N,256, H/8, W/8)
        d1_in = torch.cat([d1_up, e3], dim=1)       # (N,512, H/8, W/8)
        d1 = self.dec1_conv(d1_in)                  # (N,256, H/8, W/8)

        d2_up = self.dec2_up(d1)                    # (N,128, H/4, W/4)
        d2_in = torch.cat([d2_up, e2], dim=1)       # (N,256, H/4, W/4)
        d2 = self.dec2_conv(d2_in)                  # (N,128, H/4, W/4)

        d3_up = self.dec3_up(d2)                    # (N,64,  H/2, W/2)
        d3_in = torch.cat([d3_up, e1], dim=1)       # (N,128, H/2, W/2)
        d3 = self.dec3_conv(d3_in)                  # (N,64,  H/2, W/2)

        d4_up = self.dec4_up(d3)                    # (N,64,  H,   W)
        d4 = self.dec4_conv(d4_up)                  # (N,3,   H,   W)
        out = self.final_act(d4)
        return out

###############
# Stage2Generator (U-Net 스킵 구조 + ContextualAttention)
###############
class Stage2Generator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # ----- Encoder -----
        self.enc1 = GatedConv2d(7, 64, 4, 2, 1)
        self.enc2 = GatedConv2d(64, 128, 4, 2, 1)
        self.enc3 = GatedConv2d(128, 256, 4, 2, 1)
        self.enc4 = GatedConv2d(256, 512, 4, 2, 1)

        self.contextual_attention = ContextualAttention()

        # ----- Decoder (Skip Connection) -----
        self.dec1_up = GatedDeconv2d(512, 256, 4, 2, 1)
        self.dec1_conv = GatedConv2d(256 + 256, 256, 3, 1, 1)

        self.dec2_up = GatedDeconv2d(256, 128, 4, 2, 1)
        self.dec2_conv = GatedConv2d(128 + 128, 128, 3, 1, 1)

        self.dec3_up = GatedDeconv2d(128, 64, 4, 2, 1)
        self.dec3_conv = GatedConv2d(64 + 64, 64, 3, 1, 1)

        self.dec4_up = GatedDeconv2d(64, 64, 4, 2, 1)
        self.dec4_conv = torch.nn.Conv2d(64, 3, 3, 1, 1)
        self.final_act = torch.nn.Sigmoid()

    def forward(self, coarse_out, inp, mask):
        # (N,3,H,W), (N,3,H,W), (N,1,H,W)
        fin_inp = torch.cat([coarse_out, inp, mask], dim=1)  # (N,7,H,W)

        e1 = self.enc1(fin_inp)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)

        # Contextual Attention
        e4_attn = self.contextual_attention(e4)

        # Decoder + Skip
        d1_up = self.dec1_up(e4_attn)                 # (N,256, H/8, W/8)
        d1_in = torch.cat([d1_up, e3], dim=1)         # (N,512, H/8, W/8)
        d1 = self.dec1_conv(d1_in)

        d2_up = self.dec2_up(d1)                      # (N,128, H/4, W/4)
        d2_in = torch.cat([d2_up, e2], dim=1)         # (N,256, H/4, W/4)
        d2 = self.dec2_conv(d2_in)

        d3_up = self.dec3_up(d2)                      # (N,64, H/2, W/2)
        d3_in = torch.cat([d3_up, e1], dim=1)         # (N,128, H/2, W/2)
        d3 = self.dec3_conv(d3_in)

        d4_up = self.dec4_up(d3)                      # (N,64, H, W)
        d4 = self.dec4_conv(d4_up)                    # (N,3,  H, W)
        out = self.final_act(d4)
        return out

############################
# 가중치 경로 (새로운 학습 코드에서 생성된 best_XX)
############################
best_coarse_path = "best_coarse_generator_epoch5.pth"
best_fine_path   = "best_fine_generator_epoch5.pth"

############################
# 테스트 이미지 및 설정
############################
test_input_dir = "../02_color/data/output_grayTocol_2024123101"  # 예시
test_mask_dir  = "../data/output_01_mask"
output_dir     = "data/output_colToper_2024123101"
os.makedirs(output_dir, exist_ok=True)

coarse_generator = Stage1Generator().to(device)
fine_generator   = Stage2Generator().to(device)

# 새로 학습한 U-Net 구조 가중치 로드
coarse_generator.load_state_dict(torch.load(best_coarse_path, map_location=device))
fine_generator.load_state_dict(torch.load(best_fine_path,   map_location=device))

coarse_generator.eval()
fine_generator.eval()

transform = T.Compose([
    T.Resize((512,512)),
    T.ToTensor()
])

test_files = [f for f in os.listdir(test_input_dir) if f.lower().endswith(('.png','.jpg','.jpeg'))]

with torch.no_grad():
    for filename in test_files:
        input_path = os.path.join(test_input_dir, filename)
        mask_path = os.path.join(test_mask_dir, filename)
        
        if not os.path.exists(mask_path):
            print(f"{mask_path}가 존재하지 않습니다. 스킵합니다.")
            continue

        inp_img = Image.open(input_path).convert("RGB")
        mask_img = Image.open(mask_path).convert("L")

        inp_tensor = transform(inp_img).unsqueeze(0).to(device)    # (1,3,H,W)
        mask_tensor = transform(mask_img).unsqueeze(0).to(device)  # (1,1,H,W)

        # 손상 영역 0 처리
        mask_broadcast = mask_tensor.expand_as(inp_tensor)         # (1,3,H,W)로 확장
        damaged_inp = inp_tensor * (1.0 - mask_broadcast)

        # Stage1 (Coarse)
        coarse_out = coarse_generator(damaged_inp, mask_tensor)

        # Stage2 (Fine)
        fine_out = fine_generator(coarse_out, damaged_inp, mask_tensor)

        # 마스크 영역에 fine_out을 씌우기
        final_result = inp_tensor * (1 - mask_broadcast) + fine_out * mask_broadcast

        # 결과 저장
        final_result_pil = T.ToPILImage()(final_result.squeeze(0).cpu())
        save_path = os.path.join(output_dir, filename)
        final_result_pil.save(save_path)

        print(f"{filename} 복원 완료 -> {save_path}")


TEST_000.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_000.png
TEST_001.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_001.png
TEST_002.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_002.png
TEST_003.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_003.png
TEST_004.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_004.png
TEST_005.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_005.png
TEST_006.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_006.png
TEST_007.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_007.png
TEST_008.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_008.png
TEST_009.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_009.png
TEST_010.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_010.png
TEST_011.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_011.png
TEST_012.png 복원 완료(마스크 영역만 덮

# 마스크 확장버전

In [1]:
import os
import torch
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np
import cv2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

############################
# 모델 구조 정의 (학습시 사용한 것과 동일 - U-Net 스킵 구조)
############################

class GatedConv2d(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, activation=torch.nn.ReLU()):
        super().__init__()
        self.feature_conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.mask_conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.sigmoid = torch.nn.Sigmoid()
        self.activation = activation

    def forward(self, x):
        f = self.feature_conv(x)
        m = self.mask_conv(x)
        gated = self.sigmoid(m)
        if self.activation is not None:
            f = self.activation(f)
        return f * gated

class GatedDeconv2d(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=4, stride=2, padding=1, activation=torch.nn.ReLU()):
        super().__init__()
        self.feature_deconv = torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding)
        self.mask_deconv = torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding)
        self.sigmoid = torch.nn.Sigmoid()
        self.activation = activation

    def forward(self, x):
        f = self.feature_deconv(x)
        m = self.mask_deconv(x)
        gated = self.sigmoid(m)
        if self.activation is not None:
            f = self.activation(f)
        return f * gated

class ContextualAttention(torch.nn.Module):
    def __init__(self, kernel_size=3, stride=1, dilation=1):
        super().__init__()
        self.conv = torch.nn.Conv2d(512, 512, kernel_size, stride, dilation, bias=False)
        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, x):
        B,C,H,W = x.size()
        query = x.view(B,C,-1)
        key = x.view(B,C,-1)
        value = x.view(B,C,-1)
        attn = torch.bmm(query.permute(0,2,1), key)  # (B,H*W,C) x (B,C,H*W)
        attn = self.softmax(attn)
        out = torch.bmm(attn, value.permute(0,2,1))
        out = out.permute(0,2,1).view(B,C,H,W)
        out = self.conv(out)
        return out

###############
# Stage1Generator (U-Net 스킵 구조)
###############
class Stage1Generator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # ----- Encoder (Down) -----
        self.enc1 = GatedConv2d(4, 64, 4, 2, 1)
        self.enc2 = GatedConv2d(64, 128, 4, 2, 1)
        self.enc3 = GatedConv2d(128, 256, 4, 2, 1)
        self.enc4 = GatedConv2d(256, 512, 4, 2, 1)

        # ----- Decoder (Up) -----
        self.dec1_up = GatedDeconv2d(512, 256, 4, 2, 1)
        self.dec1_conv = GatedConv2d(256 + 256, 256, 3, 1, 1)

        self.dec2_up = GatedDeconv2d(256, 128, 4, 2, 1)
        self.dec2_conv = GatedConv2d(128 + 128, 128, 3, 1, 1)

        self.dec3_up = GatedDeconv2d(128, 64, 4, 2, 1)
        self.dec3_conv = GatedConv2d(64 + 64, 64, 3, 1, 1)

        self.dec4_up = GatedDeconv2d(64, 64, 4, 2, 1)
        self.dec4_conv = torch.nn.Conv2d(64, 3, 3, 1, 1)
        self.final_act = torch.nn.Sigmoid()

    def forward(self, x, mask):
        # Encoder
        inp = torch.cat((x, mask), dim=1)  # (N,4,H,W)
        e1 = self.enc1(inp)    # (N,64,  H/2,  W/2)
        e2 = self.enc2(e1)     # (N,128, H/4,  W/4)
        e3 = self.enc3(e2)     # (N,256, H/8,  W/8)
        e4 = self.enc4(e3)     # (N,512, H/16, W/16)

        # Decoder (Skip Connection)
        d1_up = self.dec1_up(e4)                    # (N,256, H/8, W/8)
        d1_in = torch.cat([d1_up, e3], dim=1)       # (N,512, H/8, W/8)
        d1 = self.dec1_conv(d1_in)                  # (N,256, H/8, W/8)

        d2_up = self.dec2_up(d1)                    # (N,128, H/4, W/4)
        d2_in = torch.cat([d2_up, e2], dim=1)       # (N,256, H/4, W/4)
        d2 = self.dec2_conv(d2_in)                  # (N,128, H/4, W/4)

        d3_up = self.dec3_up(d2)                    # (N,64,  H/2, W/2)
        d3_in = torch.cat([d3_up, e1], dim=1)       # (N,128, H/2, W/2)
        d3 = self.dec3_conv(d3_in)                  # (N,64,  H/2, W/2)

        d4_up = self.dec4_up(d3)                    # (N,64,  H,   W)
        d4 = self.dec4_conv(d4_up)                  # (N,3,   H,   W)
        out = self.final_act(d4)
        return out

###############
# Stage2Generator (U-Net 스킵 구조 + ContextualAttention)
###############
class Stage2Generator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # ----- Encoder -----
        self.enc1 = GatedConv2d(7, 64, 4, 2, 1)
        self.enc2 = GatedConv2d(64, 128, 4, 2, 1)
        self.enc3 = GatedConv2d(128, 256, 4, 2, 1)
        self.enc4 = GatedConv2d(256, 512, 4, 2, 1)

        self.contextual_attention = ContextualAttention()

        # ----- Decoder (Skip Connection) -----
        self.dec1_up = GatedDeconv2d(512, 256, 4, 2, 1)
        self.dec1_conv = GatedConv2d(256 + 256, 256, 3, 1, 1)

        self.dec2_up = GatedDeconv2d(256, 128, 4, 2, 1)
        self.dec2_conv = GatedConv2d(128 + 128, 128, 3, 1, 1)

        self.dec3_up = GatedDeconv2d(128, 64, 4, 2, 1)
        self.dec3_conv = GatedConv2d(64 + 64, 64, 3, 1, 1)

        self.dec4_up = GatedDeconv2d(64, 64, 4, 2, 1)
        self.dec4_conv = torch.nn.Conv2d(64, 3, 3, 1, 1)
        self.final_act = torch.nn.Sigmoid()

    def forward(self, coarse_out, inp, mask):
        # (N,3,H,W), (N,3,H,W), (N,1,H,W)
        fin_inp = torch.cat([coarse_out, inp, mask], dim=1)  # (N,7,H,W)

        e1 = self.enc1(fin_inp)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)

        # Contextual Attention
        e4_attn = self.contextual_attention(e4)

        # Decoder + Skip
        d1_up = self.dec1_up(e4_attn)                 # (N,256, H/8, W/8)
        d1_in = torch.cat([d1_up, e3], dim=1)         # (N,512, H/8, W/8)
        d1 = self.dec1_conv(d1_in)

        d2_up = self.dec2_up(d1)                      # (N,128, H/4, W/4)
        d2_in = torch.cat([d2_up, e2], dim=1)         # (N,256, H/4, W/4)
        d2 = self.dec2_conv(d2_in)

        d3_up = self.dec3_up(d2)                      # (N,64, H/2, W/2)
        d3_in = torch.cat([d3_up, e1], dim=1)         # (N,128, H/2, W/2)
        d3 = self.dec3_conv(d3_in)

        d4_up = self.dec4_up(d3)                      # (N,64, H, W)
        d4 = self.dec4_conv(d4_up)                    # (N,3,  H, W)
        out = self.final_act(d4)
        return out

############################
# 가중치 경로 (새로운 학습 코드에서 생성된 best_XX)
############################
best_coarse_path = "model/04/best_coarse_generator_epoch4 (3).pth"
best_fine_path   = "model/04/best_fine_generator_epoch4 (3).pth"

############################
# 테스트 이미지 및 설정
############################
test_input_dir = "../02_color/data/output_grayTocol_2025010201_nowbest"
test_mask_dir  = "../data/output_01_mask"
output_dir     = "data/output_colToper_2025010203"
os.makedirs(output_dir, exist_ok=True)

coarse_generator = Stage1Generator().to(device)
fine_generator   = Stage2Generator().to(device)

# 새로 학습한 U-Net 구조 가중치 로드
coarse_generator.load_state_dict(torch.load(best_coarse_path, map_location=device))
fine_generator.load_state_dict(torch.load(best_fine_path,   map_location=device))

coarse_generator.eval()
fine_generator.eval()

transform = T.Compose([
    T.Resize((512,512)),
    T.ToTensor()
])

test_files = [f for f in os.listdir(test_input_dir) if f.lower().endswith(('.png','.jpg','.jpeg'))]

with torch.no_grad():
    for filename in test_files:
        input_path = os.path.join(test_input_dir, filename)
        mask_path  = os.path.join(test_mask_dir, filename)

        if not os.path.exists(mask_path):
            print(f"{mask_path}가 존재하지 않습니다. 스킵합니다.")
            continue

        inp_img = Image.open(input_path).convert("RGB")
        mask_img = Image.open(mask_path).convert("L")

        # ========== (1) 마스크 확장(팽창, dilation) 작업 ==========
        mask_np = np.array(mask_img, dtype=np.uint8)
        
        # 커널 크기를 조절하면 마스크 확장 범위를 조절할 수 있습니다 (3x3, 5x5 등)
        kernel = np.ones((3,3), np.uint8)  
        
        # iterations=1 정도면 조금만 확장, 더 크게 하고 싶으면 2~3으로 늘려보세요
        dilated_mask = cv2.dilate(mask_np, kernel, iterations=1)
        
        # 다시 PIL Image로 변환
        mask_img = Image.fromarray(dilated_mask)

        # 변환(transform)
        inp_tensor = transform(inp_img).unsqueeze(0).to(device)    # (1,3,H,W)
        mask_tensor = transform(mask_img).unsqueeze(0).to(device)  # (1,1,H,W)

        # 손상 영역 0 처리
        mask_broadcast = mask_tensor.expand_as(inp_tensor)  # (1,3,H,W)
        damaged_inp = inp_tensor * (1.0 - mask_broadcast)

        # Stage1 (Coarse)
        coarse_out = coarse_generator(damaged_inp, mask_tensor)

        # Stage2 (Fine)
        fine_out = fine_generator(coarse_out, damaged_inp, mask_tensor)

        # 최종 복원 결과 = 원본(손상 부위 0) + (fine_out * 마스크)
        final_result = inp_tensor * (1.0 - mask_broadcast) + fine_out * mask_broadcast

        # (2) 결과 저장
        final_result_pil = T.ToPILImage()(final_result.squeeze(0).cpu())
        save_path = os.path.join(output_dir, filename)
        final_result_pil.save(save_path)

        print(f"{filename} 복원 완료 (마스크 확장 적용) -> {save_path}")

C:\Users\zqrc0\AppData\Local\Temp\ipykernel_26648\1955511395.py:193: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  coarse_generator.load_state_dict(torch.load(best_coarse_pa

TEST_000.png 복원 완료 (마스크 확장 적용) -> data/output_colToper_2024123103\TEST_000.png
TEST_001.png 복원 완료 (마스크 확장 적용) -> data/output_colToper_2024123103\TEST_001.png
TEST_002.png 복원 완료 (마스크 확장 적용) -> data/output_colToper_2024123103\TEST_002.png
TEST_003.png 복원 완료 (마스크 확장 적용) -> data/output_colToper_2024123103\TEST_003.png
TEST_004.png 복원 완료 (마스크 확장 적용) -> data/output_colToper_2024123103\TEST_004.png
TEST_005.png 복원 완료 (마스크 확장 적용) -> data/output_colToper_2024123103\TEST_005.png
TEST_006.png 복원 완료 (마스크 확장 적용) -> data/output_colToper_2024123103\TEST_006.png
TEST_007.png 복원 완료 (마스크 확장 적용) -> data/output_colToper_2024123103\TEST_007.png
TEST_008.png 복원 완료 (마스크 확장 적용) -> data/output_colToper_2024123103\TEST_008.png
TEST_009.png 복원 완료 (마스크 확장 적용) -> data/output_colToper_2024123103\TEST_009.png
TEST_010.png 복원 완료 (마스크 확장 적용) -> data/output_colToper_2024123103\TEST_010.png
TEST_011.png 복원 완료 (마스크 확장 적용) -> data/output_colToper_2024123103\TEST_011.png
TEST_012.png 복원 완료 (마스크 확장 적용) -> data/output_colTop